## Imports and libraries installation

In [ ]:
%%capture
import os
import cv2 
import ast
import time
import json
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader

# install libraries
!pip install wandb -qqq
import wandb
!pip install -U git+https://github.com/qubvel/segmentation_models.pytorch
import segmentation_models_pytorch as smp


## Install custom scripts

In [ ]:
!git clone https://github.com/AGiannoutsos/End-to-End-Lane-Detection
%cd End-to-End-Lane-Detection

## Weights and biases initialization

In [ ]:
import wandb
os.environ["WANDB_ENTITY"]  = "andreas_giannoutsos"
os.environ["WANDB_PROJECT"] = "lane_detection"
os.environ["WANDB_RESUME"]  = "allow"
from scripts.wandb_util import *

## Dataset api from the stored files in the hardrive to arrays with label images

In [ ]:
from scripts.dataset import *

## Fuctions that store a video clip from the given images

In [ ]:
from scripts.visualization import *

## Download, load, transform and save data images (done once)

In [ ]:
# Download TUsimple dataset
download_TUsimple_dataset("train")

# Load and transform images and labels for classic lane detector
train_dataset = Dataset("train_set", (128,128))

# Load, transform and save the dataset optimized for pytroch models
pytorch_train_dataset = Pytorch_Dataset("pytorch_train_set", train_dataset)

# Upload dataset to weighs and biases cloud as an artifact so that we wont need to make these transforms again
wandb_log_artifact(artifact_name="tusimple_train_set_1x128x128", directory="pytorch_train_set", type_="dataset")

## Download transformed data from cloud and make sampler for train and validation set

In [ ]:
# Load dataset as artifact from weights and biases
os.environ["WANDB_ENTITY"]  = "andreas_giannoutsos"
os.environ["WANDB_PROJECT"] = "lane_detection"
os.environ["WANDB_RESUME"]  = "allow"
artifact_directory = wandb_load_artifact(artifact_name="tusimple_train_set_1x128x128", version="latest")

# Load data optimized for pytorch models
pytorch_train_dataset = Pytorch_Dataset(artifact_directory)

# set a sampler for train and validation dataset
# first 10% of training set will be validation set
split = int(len(pytorch_train_dataset)*0.1)
indices = list(range(len(pytorch_train_dataset)))
np.random.shuffle(indices)
train_sampler      = torch.utils.data.sampler.SubsetRandomSampler(indices[split:])
validation_sampler = torch.utils.data.sampler.SubsetRandomSampler(indices[0:split])

## Autoencoder model definition

In [ ]:
import torch
import torch.nn as nn

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print(device)

cuda


In [ ]:
class Model(nn.Module):

    def __init__(self, module):
        super(Model, self).__init__()
        self.module = module
    
    def get_output(self, image):
        with torch.no_grad():
            output = self(image.unsqueeze(0)).cpu()
            return torch_model_to_cv2(output[0])

    def forward(self, x):
        x = self.module(x)              
        return x

## Train methods

In [ ]:
from scripts.train_methods import *

## Simple vanilla Auto-Encoder 0.5M parameters

In [ ]:
class Config(object):
    def __init__(self):
        # parameters
        self.learning_rate   = 1e-3
        self.dropout         = 0
        self.weight_decay    = 0.0
        self.gradient_clip   = 1
        self.batch_size      = 32
        self.val_batch_size  = 32
        self.epochs          = 1
        self.loss            = nn.BCELoss()
    
WANDB_ID            = "simple_autoencoder_t3"
WNDB_NAME           = "simple_autoencoder_t3"
LOAD_SAVED_MODEL    = False
LOG_INTERVAL        = 5
MODEL_SAVE_NAME     = "simple_autoencoder_t3"
SAVED_MODEL_VERSION = "latest"

config = Config()
train_dataloader      = DataLoader(pytorch_train_dataset, batch_size=config.batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=train_sampler)
validation_dataloader = DataLoader(pytorch_train_dataset, batch_size=config.val_batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=validation_sampler)

SimpleAutoEncoder = nn.Sequential(
          nn.Conv2d(1, 8, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(8),
          nn.Conv2d(8, 16, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.MaxPool2d(2, 2),

          nn.Conv2d(16, 32, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Conv2d(32, 64, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(64),
          nn.MaxPool2d(2, 2),

          nn.Conv2d(64, 128, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(128),
          nn.Conv2d(128, 128, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(128),
          nn.MaxPool2d(2, 2),

          nn.Conv2d(128, 128, 3, padding=1),
          nn.ReLU(),
          nn.Conv2d(128, 64, 3, padding=1),
          nn.ReLU(),
          nn.Upsample(scale_factor=2, mode='nearest'),

          nn.Conv2d(64, 32, 3, padding=1),
          nn.ReLU(),
          nn.Conv2d(32, 16, 3, padding=1),
          nn.ReLU(),
          nn.Upsample(scale_factor=2, mode='nearest'),

          nn.Conv2d(16, 8, 3, padding=1),
          nn.ReLU(),
          nn.Conv2d(8, 1, 3, padding=1),
          nn.Upsample(scale_factor=2, mode='nearest'),
          nn.Sigmoid()
        )

if ('model' not in globals()) and (LOAD_SAVED_MODEL is False): # not reinitializing the model or the optimizer
    print("INITIALIZE NEW MODEL")
    model = Model(SimpleAutoEncoder).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay) # optimizer

elif LOAD_SAVED_MODEL is True: # load model from cloud
    try:
        print("LOAD SAVED MODEL")
        model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
        # load model, history, optimizer
        checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
        initial_epoch = checkpoint["initial_epoch"]
        history = checkpoint["history"]
        model = checkpoint["model"].to(device)
        optimizer = checkpoint['optimizer_state_dict']
    except:
        print("NO MODEL FOUND")


run = wandb.init(config=config.__dict__, resume=WANDB_ID)  
run.name = WNDB_NAME


history = train_model(model, optimizer, config.loss, config, train_dataloader, validation_dataloader, device, checkpoint_path=MODEL_SAVE_NAME, run=run, log_interval=LOG_INTERVAL, verbose=0)

In [ ]:
del model

## Unet + ResNet34 21M parameters

In [ ]:
class Config(object):
    def __init__(self):
        # parameters
        self.learning_rate   = 1e-4
        self.dropout         = 0
        self.weight_decay    = 0.0
        self.gradient_clip   = 1
        self.batch_size      = 32
        self.val_batch_size  = 32
        self.epochs          = 5
        self.loss            = nn.BCELoss()
    
WANDB_ID            = "unet_resnet34_t3"
WNDB_NAME           = "unet_resnet34_t3"
LOAD_SAVED_MODEL    = True
LOG_INTERVAL        = 5
MODEL_SAVE_NAME     = "unet_resnet34_t3"
SAVED_MODEL_VERSION = "latest"

config = Config()
train_dataloader      = DataLoader(pytorch_train_dataset, batch_size=config.batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=train_sampler)
validation_dataloader = DataLoader(pytorch_train_dataset, batch_size=config.val_batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=validation_sampler)

if ('model' not in globals()) and (LOAD_SAVED_MODEL is False): # not reinitializing the model or the optimizer
    print("INITIALIZE NEW MODEL")
    model = Model(smp.Unet(encoder_name='resnet34',
                            encoder_depth=5,
                            encoder_weights='imagenet',
                            decoder_use_batchnorm=True,
                            decoder_channels=(256, 128, 64, 32, 16),
                            decoder_attention_type=None,
                            in_channels=1,
                            classes=1,
                            activation="sigmoid",
                            aux_params=None)
                ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay) # optimizer

elif LOAD_SAVED_MODEL is True: # load model from cloud
    try:
        print("LOAD SAVED MODEL")
        model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
        # load model, history, optimizer
        checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
        initial_epoch = checkpoint["initial_epoch"]
        history = checkpoint["history"]
        model = checkpoint["model"].to(device)
        optimizer = checkpoint['optimizer_state_dict']
    except:
        print("NO MODEL FOUND")


run = wandb.init(config=config.__dict__, resume=WANDB_ID)  
run.name = WNDB_NAME


history = train_model(model, optimizer, config.loss, config, train_dataloader, validation_dataloader, device, checkpoint_path=MODEL_SAVE_NAME, run=run, log_interval=LOG_INTERVAL, verbose=0)


In [ ]:
del model

## Unet + MobileNetV2 2M parameters

In [ ]:
class Config(object):
    def __init__(self):
        # parameters
        self.learning_rate   = 1e-4
        self.dropout         = 0
        self.weight_decay    = 0.0
        self.gradient_clip   = 1
        self.batch_size      = 32
        self.val_batch_size  = 32
        self.epochs          = 3
        self.loss            = nn.BCELoss()
    
WANDB_ID            = "unet_mobilenetv2_t1"
WNDB_NAME           = "unet_mobilenetv2_t1"
LOAD_SAVED_MODEL    = True
LOG_INTERVAL        = 5
MODEL_SAVE_NAME     = "unet_mobilenetv2_t1"
SAVED_MODEL_VERSION = "latest"

config = Config()
train_dataloader      = DataLoader(pytorch_train_dataset, batch_size=config.batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=train_sampler)
validation_dataloader = DataLoader(pytorch_train_dataset, batch_size=config.val_batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=validation_sampler)

if ('model' not in globals()) and (LOAD_SAVED_MODEL is False): # not reinitializing the model or the optimizer
    print("INITIALIZE NEW MODEL")
    model = Model(smp.Unet(encoder_name='mobilenet_v2',
                            encoder_depth=5,
                            encoder_weights='imagenet',
                            decoder_use_batchnorm=True,
                            decoder_channels=(256, 128, 64, 32, 16),
                            decoder_attention_type=None,
                            in_channels=1,
                            classes=1,
                            activation="sigmoid",
                            aux_params=None)
                ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay) # optimizer

elif LOAD_SAVED_MODEL is True: # load model from cloud
    try:
        print("LOAD SAVED MODEL")
        model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
        # load model, history, optimizer
        checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
        initial_epoch = checkpoint["initial_epoch"]
        history = checkpoint["history"]
        model = checkpoint["model"].to(device)
        optimizer = checkpoint['optimizer_state_dict']
    except:
        print("NO MODEL FOUND")


run = wandb.init(config=config.__dict__, resume=WANDB_ID)  
run.name = WNDB_NAME


history = train_model(model, optimizer, config.loss, config, train_dataloader, validation_dataloader, device, checkpoint_path=MODEL_SAVE_NAME, run=run, log_interval=LOG_INTERVAL, verbose=0)


In [ ]:
del model

## Unet + EfficientNet 6M parameters

In [ ]:
class Config(object):
    def __init__(self):
        # parameters
        self.learning_rate   = 1e-4
        self.dropout         = 0
        self.weight_decay    = 0.0
        self.gradient_clip   = 1
        self.batch_size      = 32
        self.val_batch_size  = 32
        self.epochs          = 7
        self.loss            = nn.BCELoss()
    
WANDB_ID            = "unet_efficentnet_t1"
WNDB_NAME           = "unet_efficentnet_t1"
LOAD_SAVED_MODEL    = True
LOG_INTERVAL        = 5
MODEL_SAVE_NAME     = "unet_efficentnet_t1"
SAVED_MODEL_VERSION = "latest"

config = Config()
train_dataloader      = DataLoader(pytorch_train_dataset, batch_size=config.batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=train_sampler)
validation_dataloader = DataLoader(pytorch_train_dataset, batch_size=config.val_batch_size, pin_memory=True, num_workers=1, drop_last=True, sampler=validation_sampler)

if ('model' not in globals()) and (LOAD_SAVED_MODEL is False): # not reinitializing the model or the optimizer
    print("INITIALIZE NEW MODEL")
    model = Model(smp.Unet(encoder_name='efficientnet-b1',
                            encoder_depth=5,
                            encoder_weights='imagenet',
                            decoder_use_batchnorm=True,
                            decoder_channels=(256, 128, 64, 32, 16),
                            decoder_attention_type=None,
                            in_channels=1,
                            classes=1,
                            activation="sigmoid",
                            aux_params=None)
                ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay) # optimizer

elif LOAD_SAVED_MODEL is True: # load model from cloud
    try:
        print("LOAD SAVED MODEL")
        model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
        # load model, history, optimizer
        checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
        initial_epoch = checkpoint["initial_epoch"]
        history = checkpoint["history"]
        model = checkpoint["model"].to(device)
        optimizer = checkpoint['optimizer_state_dict']
    except:
        print("NO MODEL FOUND")


run = wandb.init(config=config.__dict__, resume=WANDB_ID)  
run.name = WNDB_NAME


history = train_model(model, optimizer, config.loss, config, train_dataloader, validation_dataloader, device, checkpoint_path=MODEL_SAVE_NAME, run=run, log_interval=LOG_INTERVAL, verbose=0)


In [ ]:
del model

In [ ]:
autoencoder_visualization([history], pytorch_train_dataset, model, device)

In [ ]:
# pytorch_train_dataset[0][0]
# model(pytorch_train_dataset[0][0].unsqueeze(0))
# pytorch_train_dataset[0][1].max()
import pandas as pd
# out.mean()
df = pd.DataFrame(out.ravel())
df.value_counts()

0      49104
1         36
255       12
dtype: int64

In [ ]:
# a = torch.where(pytorch_train_dataset[0][1] > pytorch_train_dataset[0][1].mean(), 1.0, 0.0)
# a.max()
del model

In [ ]:
# for data, labels in train_dataloader:
#     out = model(data)
#     break

i = 15090
out = 0
# with torch.no_grad():
    # model.eval()
out = model.get_output(pytorch_train_dataset[i][0].to(device))
out = np.where(out > out.mean(), 255,1)

cv2_imshow(out)
# cv2_imshow(cv2.resize(out, (256,256)))
# cv2_imshow(cv2.resize(train_dataset[i][0], (256,256)))
cv2_imshow(cv2.resize(torch_model_to_cv2(pytorch_train_dataset[i][0]), (256,256)))
cv2_imshow(cv2.resize(torch_model_to_cv2(pytorch_train_dataset[i][1]), (256,256)))
cv2_imshow(cv2.resize(torch_model_to_cv2(a), (256,256)))

In [ ]:
torch_video_detector_creator("model", model, train_dataset, pytorch_train_dataset, 0, 99, True, fps=24)
# video_detector_creator("model", detector, train_dataset, 0, 99, True, fps=24)

In [ ]:
from moviepy.editor import *
path = "model.avi" 
clip=VideoFileClip(path)
clip.ipython_display(width=512)

## Results get the detectors

In [ ]:
MODEL_SAVE_NAME     = "simple_autoencoder_t2"
SAVED_MODEL_VERSION = "latest"
model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
simple_autoencoder_model = checkpoint["model"].to(device)

MODEL_SAVE_NAME     = "unet_resnet34_t3"
SAVED_MODEL_VERSION = "latest"
model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
resNet34_model = checkpoint["model"].to(device)

MODEL_SAVE_NAME     = "unet_mobilenetv2_t1"
SAVED_MODEL_VERSION = "latest"
model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
mobileNetV2_model = checkpoint["model"].to(device)

MODEL_SAVE_NAME     = "unet_efficentnet_t1"
SAVED_MODEL_VERSION = "latest"
model_artifact_directory = wandb_load_artifact(run, MODEL_SAVE_NAME, SAVED_MODEL_VERSION)
checkpoint = torch.load(os.path.join(model_artifact_directory,MODEL_SAVE_NAME))
efficientNet_model = checkpoint["model"].to(device)

detectors = [simple_autoencoder_model,
             resNet34_model,
             mobileNetV2_model,
             efficientNet_model]
texts =     ["Simple AE 0.5M",
             "ResNet 21M",
             "MobileNet 2M",
             "EfficientNet 6M"]


## Create grid videos

In [ ]:
torch_grid_video_detector_creator("video_AEmodels_grid",
                  detectors,
                  texts,
                  pytorch_train_dataset, 
                  70000, 
                  2000,
                  grid=(2,2), 
                  labels=False, 
                  fps=24, 
                  overlay_opacity=0.2)

## Create grid images

In [ ]:
for i in range(10):
    torch_grid_image_detector_creator("image"+str(i)+"_AEmodels_grid.jpg",
                  detectors,
                  texts,
                  pytorch_train_dataset, 
                  np.random.randint(len(pytorch_train_dataset)), 
                  grid=(2,2), 
                  labels=False, 
                  fps=24, 
                  overlay_opacity=0.2)

In [ ]:
from moviepy.editor import *
path = "video_AEmodels_grid.avi" 
clip=VideoFileClip(path)
clip.ipython_display(width=512)